### Small Tensorflow project using MNIST dataset.

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

##### Load MNIST dataset

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale to range 0..1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Add channels
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

##### Shuffle dataset and prepare batches

In [10]:
training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
training_dataset = training_dataset.shuffle(buffer_size=training_dataset.cardinality()).batch(32)

testing_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

##### Build a model

In [ ]:
class model_1conv(Model):
    # Constructor
    def __init__(self, n_of_filters, n_of_dense_neurons):
        super(model_1conv, self).__init__()

        self.conv = Conv2D(filters=n_of_filters,
                           kernel_size=3,
                           activation="relu")
        self.flatten = Flatten()
        self.dense_1 = Dense(n_of_dense_neurons, activation="relu")
        self.dense_2 = Dense(10)

    # Call method
    def call(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dense_2(x)

        return x